In [1]:
import torch
from torch import nn

In [2]:
x = torch.randn(100, 3)
layer = nn.Linear(3, 5)
print(layer(x).shape)
print(layer.weight)
print(layer.bias)

torch.Size([100, 5])
Parameter containing:
tensor([[-0.1108, -0.4514, -0.4163],
        [ 0.3495,  0.0874,  0.4593],
        [ 0.1155,  0.4322, -0.3844],
        [-0.1602,  0.2817, -0.5329],
        [-0.3002,  0.3061, -0.4402]], requires_grad=True)
Parameter containing:
tensor([-0.3681, -0.1281,  0.4130, -0.5634, -0.4491], requires_grad=True)


In [6]:
x = torch.randn(2, 5)
layer = nn.ReLU() # 음수는 0 양수는 linear
print(x)
print(layer(x))

tensor([[ 0.2882,  1.3782,  0.4333,  0.0688,  0.8333],
        [-1.2900,  0.2007, -1.1845, -0.6450, -1.0957]])
tensor([[0.2882, 1.3782, 0.4333, 0.0688, 0.8333],
        [0.0000, 0.2007, 0.0000, 0.0000, 0.0000]])


In [9]:
x = torch.randn(3, 7) # 어떤 노드를 통과된 상태라 하자. 3개, 7 feature
drop = nn.Dropout(p = 0.3) # p는 죽일 확률!! 논문에선 살릴 확률이였다.
print(drop(x)) # 각 개가 통과마다 다르게 drop 된다..!

tensor([[ 0.3459, -1.5823,  2.9730,  1.2473, -0.0000, -1.0688,  1.3578],
        [-0.6621,  0.0000,  2.5391,  3.8966, -0.0255, -0.6990, -0.0000],
        [-0.3712, -0.1636, -0.0000, -1.2739,  0.7860,  1.0813,  0.5819]])


In [2]:
# dropout은 정말 train, test 다르게 작동하나??

class sample_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.drop_layer = nn.Sequential(nn.Linear(5, 7),
                                        nn.Dropout(p = 0.3))
    def forward(self, x):
        x = self.drop_layer(x)
        return x

model = sample_model()
model.train()
x = torch.randn(3, 5)
print(model(x)) # 0.7의 확률로 죽임

model.eval()
print(model(x)) # 다 살려 그 대신 각 노드마다 0.7 곱

tensor([[-2.0236, -0.8471, -0.0000, -1.6398, -0.0000, -0.1545,  0.0000],
        [-0.0000,  0.3171, -0.2573, -0.1815,  0.5417, -0.1272, -0.2058],
        [-0.0119,  0.0000, -0.8282, -0.2105, -0.0000, -0.1014,  0.0000]],
       grad_fn=<MulBackward0>)
tensor([[-1.4165, -0.5930, -1.1434, -1.1478, -1.6408, -0.1081,  0.9608],
        [-0.1661,  0.2220, -0.1801, -0.1270,  0.3792, -0.0891, -0.1441],
        [-0.0084,  0.1061, -0.5797, -0.1474, -0.3469, -0.0710,  0.2844]],
       grad_fn=<AddmmBackward0>)


In [9]:
layer = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=3, stride=1, padding=1) # stride =1, padding =0이 디폴트
'''
in_channel == 필터의 채널 수
out_chanel == 필터의 수 즉, 출력되는 데이터의 채널의 수
'''
print(layer(torch.randn(32, 1, 5, 5)).shape) # 개체행렬!
# nn.linear(3, 5) # 채 채 -> 이놈은 채 또는 개체를 원함 -> 즉, 개 x 3이야지 통과 가능하다.
# nn.Conv2d(3, 5) # 채 채 -> 애는 채 행 열 또는 개 채 행 열을 원함, 개 x 3 x 행 x 열

torch.Size([32, 2, 5, 5])


In [12]:
layer = nn.Conv2d(3, 5, 3, stride =2 , padding =1)
print(layer(torch.randn(32, 3, 5, 5)).shape)
print(layer.weight.shape) # 개 채 행 렬 -> 3 x 3 filter가 3의 channel을 가지고 총 5개가 있다.

torch.Size([32, 5, 3, 3])
torch.Size([5, 3, 3, 3])


In [15]:
conv1 = nn.Conv2d(1, 8, 6, stride=2)
x = torch.randn(32, 1, 28, 28)
print(conv1(x).shape)

conv2 = nn.Conv2d(8, 16, 3, padding=1)
print(conv2(conv1(x)).shape)

Maxpool = nn.MaxPool2d(kernel_size=2, stride=(2, 2))
print(Maxpool(conv2(conv1(x))).shape)

torch.Size([32, 8, 12, 12])
torch.Size([32, 16, 12, 12])
torch.Size([32, 16, 6, 6])


In [18]:
maxpool = nn.MaxPool2d(2) # 2 로만 줘도 자동 kernel_size = 2, stride = (2, 2)
x = torch.randn(1, 1, 6, 6)
print(x)
print(maxpool(x))

tensor([[[[ 0.3499, -0.3534,  0.4921, -0.0284, -0.2667, -1.0244],
          [ 0.9035,  1.5047,  0.2959,  1.3930,  0.5262, -0.0326],
          [ 1.1675, -0.7535,  0.1152, -1.1356, -0.4297,  0.2672],
          [-1.2772,  0.9734,  0.3927, -0.1595,  0.4006,  0.2541],
          [ 1.3374, -1.6176, -0.1437, -0.5672,  0.1777,  0.6217],
          [-0.6175,  1.3684,  2.3634,  0.0053,  0.9664, -1.2419]]]])
tensor([[[[1.5047, 1.3930, 0.5262],
          [1.1675, 0.3927, 0.4006],
          [1.3684, 2.3634, 0.9664]]]])


In [19]:
avgpool = nn.AvgPool2d(2)
x = torch.randn(1, 1, 6, 6)
print(x)
print(avgpool(x))

tensor([[[[ 0.7793,  0.1524,  0.2603, -0.7056,  0.6494,  0.7938],
          [ 1.0123, -0.2509, -1.0570, -0.3321, -0.7396,  0.2015],
          [-1.6956,  0.3449, -0.4866, -0.8647, -0.6779,  1.0431],
          [-0.1455, -1.5248,  0.3526, -1.1165, -1.4569,  1.8551],
          [ 0.6834, -0.1479,  1.4771,  0.0593,  0.8920, -0.5520],
          [-0.6460,  1.0205, -1.6543,  0.0640,  0.6712,  0.5071]]]])
tensor([[[[ 0.4233, -0.4586,  0.2263],
          [-0.7552, -0.5288,  0.1909],
          [ 0.2275, -0.0135,  0.3796]]]])


In [21]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 8, 6, stride=2)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.Maxpool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(16 * 6 * 6, 10) # 잘모르겠으면 Flatten 이건까지만 하고 실행

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.Maxpool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

x = torch.randn(32, 1, 28, 28)
model = CNN()
print(model(x).shape)

torch.Size([32, 10])
